# EDA students academic performance in math.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from itertools import combinations
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency

In [ ]:
stud_math = pd.read_csv('stud_math.csv')

## Initial data inspection

In [ ]:
stud_math.head(15)

In [ ]:
stud_math.info()

In [ ]:
# Checking duplicates in dataframe.
stud_math.loc[stud_math.duplicated() == True]

## Preprocessing

In [ ]:
# 1. Make a pattern of primary look on columns in dataframe (info, counts and so on).


def get_describe(column):

    display(stud_math[column].value_counts())
    display(stud_math[column].describe())
    nul = stud_math[column].isna().sum()
    mode = stud_math[column].mode()[0]
    unique = stud_math[column].nunique()

    print('Уникальных значений:  {},'.format(unique), 'Чаще всего встречается значение: {},'.format(mode),
          'Пустых значений: {}'.format(nul))

# 2. Create function to get outliers from quantitative variables distribution.


def get_outliers(column):

    Q1 = stud_math[column].quantile(q=0.25, interpolation='midpoint')
    Q3 = stud_math[column].quantile(q=0.75, interpolation='midpoint')
    IQR = Q3 - Q1
    W1 = stud_math[stud_math[column] < Q1 -
                   1.5*IQR][column]  # bottom outlier series
    W3 = stud_math[stud_math[column] > Q3 +
                   1.5*IQR][column]  # upper outlier series

    if (len(W1.value_counts()) == 0) & (len(W3.value_counts()) == 0):
        print('There is no outliers')
    elif len(W1.value_counts()) > 0:
        print('There are bottom outliers:', W1.values)
    elif len(W3.value_counts()) > 0:
        print('There are upper outliers:', W3.values)


# 3. Create function to check Chi2 method between category variables.
# df = dataframe, col1,col2 = columns which we want to get dependence, alpha = level of method,
# all_rel = 1 - we want to see all H0 and H1 relations, 0 - to see only H1 dependent.

def get_chi2(df, col1, col2, alpha, all_rel):

    # First make a pivot table with needed variables.
    pivot = df.pivot_table(values=['score'], index=[col1], columns=[col2],
                           aggfunc='count', fill_value=0)

    # Method Chi2: H0 - variables are independent. H1 - variables are dependent.
    # We need to fill list of lists - chi2_list, from pivot above.

    row_num = df[col1].nunique()
    col_num = df[col2].nunique()
    chi2_list = []
    pivot_val = 0

    for i in range(0, row_num):
        row_list = []
        for j in range(0, col_num):
            try:
                # pivot.iat extract one value from each position in out pivot table.
                pivot_val = pivot.iat[i, j]
            except:
                pivot_val = -99
                print("\n В признаке {} при индексах {}, {} возникает следующая ошибка: ".format(col2, i, j),
                      sys.exc_info()[1])
            if pivot_val != -99:
                row_list.append(pivot_val)

        chi2_list.append(row_list)

    # Getting parameters of Chi2- method and checking the hypothesis.

    stat, p, dof, expected = chi2_contingency(chi2_list)

    if all_rel == 1:
        display(pivot)
        if p <= alpha:
            print("p value is " + str(p))
            print('Зависимы (H1 true)')
        else:
            print("p value is " + str(p))
            print('Независимы (H0 true)')
    elif all_rel == 0:
        if p <= alpha:
            display(pivot)
            print("p value is " + str(p))
            print('Зависимы (H1 true)')

# 4. Create function to check current column on relation
# with all other category columns in dataframe with Chi2-method.


def get_relation(df, column, all_rel):

    df_cat = df.drop(['age', 'absences', 'score', column], axis=1)

    for col in df_cat.columns:
        get_chi2(df, column, col, 0.05, all_rel)


# 5. Create function of getting boxplots, where y='score', x='current column'

def get_boxplot(df, column):
    fig, ax = plt.subplots(figsize=(14, 4))
    sns.boxplot(x=column, y='score',
                data=df,
                ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()


# 6. Create function of checking T-test.

def get_ttest(df, column):
    cols = df.loc[:, column].value_counts().index
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        p_test = ttest_ind(df.loc[df.loc[:, column] == comb[0], 'score'],
                           df.loc[df.loc[:, column] == comb[1], 'score']).pvalue
        if p_test <= 0.05/len(combinations_all):
            print('p_value = {}'.format(p_test * len(combinations_all)))
            print('Найдены статистически значимые различия для столбца', column)
            break

## Primary columns analysis

### Количественные переменные

#### Age

In [ ]:
get_describe('age')

In [ ]:
stud_math.age.plot(kind='hist', grid=True, title='Age')

Из гистограммы видно, что есть выбросы, найдем их более точно с помощью функции нахождения выбросов (из блока предобработки):

In [ ]:
get_outliers('age')

Итак, уникальных значений возрастов 8, из которых самый встречающийся это 16 лет. Очень малое количество студентов от 20 до 22х лет, скорее всего это второгодки, кто не может сдать экзамен (проверим это дальше). Исходя их метода интерквартильного расстояния имеется один выброс в 22 года, но по здравому смыслу это никак не помешает дальнейшему исследованию, тем более, что разница от предыдущего возраста всего 1 год (если было бы 50 лет, мы бы выкинули его из статистики).

#### Absences

In [ ]:
get_describe('absences')

In [ ]:
stud_math.absences.plot(kind='hist', grid=True, title='Absences')

Всего 36 уникальных значений пропусков школы. Имеется 12 незаполненных значений. 
По гистограмме мы видим, что есть большие выбросы в 385 и 212 дней.
Школьных календарных дней не более 180, поэтому два значения в 385 и 212 нужно точно исключить или заменить.
Проверим с помощью функции какие точно получаются выбросы:

In [ ]:
get_outliers('absences')

По формуле интерквартильного расстояния у нас верхние выбросы начинатся с 21, просто потому что самое большое число студентов не пропускали занятия (0 встречается 111 раз). По здравому смыслу мы не будем убирать все данные, кроме тех, что противоречат логике - это 212 и 385. Посмотрим строки датафрейма, где находятся эти два значения. Заменим 212 и 385 на медиану, так как среднее искажено в большую сторону из-за этих двух значений.

In [ ]:
stud_math.loc[(stud_math.absences == 212) | (stud_math.absences == 385)][['reason', 'traveltime', 'studytime', 
                                                                          'failures','schoolsup', 'paid', 
                                                                          'activities', 'higher', 'romantic', 
                                                                          'famrel', 'freetime', 'goout',
                                                                          'Medu', 'Fedu', 'health',
                                                                          'Pstatus', 'absences', 'score']]

Сравним различающиеся показатели этих обоих учеников:

- Ученик 222 судя по данным учится очень хорошо (оценка 85 баллов) с поддержкой школы, но без платных услуг, возможно он один из лучших учеников - стипендиатов, не выходит гулять с друзьями (goout=1), поэтому заменим пропуски занятий на 0. Возможно просто кто-то пошутил, занося такое большое количество пропусков (завистливые одноклассники).

- Ученик 390 имеет 2 промаха (failures), без поддержки школы, но платит за доп мат занятия, родители в разводе, много гуляет (goout=4), оценка по мат-ке меньше среднего (45 баллов) и то за счет доп занятий скорее всего, поэтому заменим его кол-во пропусков на 21. Скорее всего допустили ошибку в указании данных (лишняя двойка в конце), при этому 21 это больше среднего значения, но не намного (как максимальное 75 из остальных имеющихся) - делаем таким образом сглаживание.

In [ ]:
stud_math.absences = stud_math.absences.replace(385, 0)
stud_math.absences = stud_math.absences.replace(212, 21)

In [ ]:
# Проверяем изменения.
stud_math.absences.plot(kind='hist', grid=True, bins=20, title='Absences')
stud_math.absences.describe()

Посмотрим строки, где в столбце absence стоят пропуски:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.absences))][['age', 'reason', 'traveltime', 'studytime', 'failures',
                                                        'schoolsup', 'paid', 'activities', 'higher', 
                                                        'romantic', 'famrel', 'freetime', 'goout',
                                                        'Medu', 'Fedu', 'Pstatus', 'health', 'score']]

Сравним показатели этих учеников:

- Ученик 150 сразу бросается в глаза с нулевой оценкой по мат-ке: он мало учится, много гуляет, имеет 3 промаха, без поддержки, не хочет иметь высшее образование, имеет романтичесике отношения, родители с начальным образованием - по всем показателям он ярый прогульщик. Заменим его absence на max=75 и посмотрим на учеников с самыми большими пропусками.

In [ ]:
stud_math.loc[150, 'absences'] = 75
stud_math.loc[stud_math.absences == 75][['age', 'reason', 'traveltime', 'studytime', 'failures',
                                         'schoolsup', 'paid', 'activities', 'higher',
                                         'romantic', 'famrel', 'freetime', 'goout',
                                         'Medu', 'Fedu', 'Pstatus', 'health', 'score']]

Можем видеть сходства: во времени на образование, отсутствие доп курсов и тп. Оставляем.

Следующий ученик, который бросается в глаза это тот, у кого не проставлена оценка за экзамен по мат-ке.

- Ученик 270: судя по тому,что он не особо много тратит времени на учебу, ходит гулять, имеет хорошее здоровье и activities, есть промахи (2), скорее всего он физически активен и возможно попадал в драки. Родители окончили школу, но хотят, чтобы ребенок все-таки поступил в университет (уже 19 лет, возможно оставался на 2й год), оплатили курсы по мат-ке. Так что score у него скорее всего ниже среднего, а absences выше среднего - заменим на 75%квартиль.

In [ ]:
stud_math.loc[270, 'absences'] = stud_math.absences.quantile(0.75)

Посмотрим на пропуски занятий тех учеников, у кого оценка меньше средней, но больше 40 (в нашей выборке это минимальное значение score):

In [ ]:
stud_math.loc[stud_math.score.between(40, stud_math.score.mean())]['absences'].value_counts()

In [ ]:
stud_math.loc[stud_math.score.between(
    40, stud_math.score.mean())]['absences'].hist()
stud_math.loc[stud_math.score.between(
    40, stud_math.score.mean())]['absences'].describe()

Среднее значение пропусков 7.66, округляем и заменяем на 8 (строки 24, 172, 352):

In [ ]:
stud_math.loc[[24, 172, 352], 'absences'] = 8

Теперь посмотрим на оставшихся учеников, у кого score выше среднего и до 90 (в нашей выборке это максимальное значени). Посмотрим на их пропуски занятий:

In [ ]:
stud_math.loc[stud_math.score.between(
    stud_math.score.mean(), 90)]['absences'].value_counts()

In [ ]:
stud_math.loc[stud_math.score.between(
    stud_math.score.mean(), 90)]['absences'].hist()
stud_math.loc[stud_math.score.between(
    stud_math.score.mean(), 90)]['absences'].describe()

Среднее 5.36, но в этом случае более хороших оценок по математике можно взять и медиану - 4 пропуска. Заменим наши оставшиеся пустые значения на медиану:

In [ ]:
stud_math.loc[[101, 120, 129, 215, 227, 254, 265], 'absences'] = stud_math.absences.median()

Проверим не пропустили ли мы какую-либо замену, не остались ли пустые значения в столбце absence:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.absences))]

Пустых значений не осталось, посмотрим теперь на итоговые исправленные данные absence:

In [ ]:
stud_math.absences.plot(kind='hist', grid=True, bins=20, title='Absences')
stud_math.absences.describe()

#### Score

In [ ]:
get_describe('score')

In [ ]:
stud_math.score.plot(kind='hist', grid=True, title='Score')

Score - оценка за экзамен по математике по 100 бальной шкале (как видим из значений). Всего уникальных оценок 18. Пустых значений 6. Самая распространенная это 50 баллов. Смущает, что достаточно много оценок равных 0 - 37 человек, либо это ученики не пришедшие на экзамен вовсе или сдавшие пустой бланк (по разным причинам: от здоровья, что-то случилось в семье, просто не хотят сдавать или поступать в высшее уч. заведение), либо не решившие ни одного задания. На этих учеников можно посмотреть отдельно - они кажутся выбросами, проверим это функцией нахождения выбросов.

In [ ]:
get_outliers('score')

Выбросов не оказалось, значит оставляем все значения как есть. Давайте посмотрим на датафрейм нулевых оценок:

In [ ]:
stud_math.loc[stud_math.score == 0][['reason', 'traveltime', 'studytime', 'failures',
                                     'schoolsup', 'paid', 'activities', 'higher',
                                     'romantic', 'famrel', 'freetime', 'goout',
                                     'Medu', 'Fedu', 'health', 'absences', 'score']][:15]

У всех учеников с нулевыми оценками число прогулов уроков тоже равно 0 (за исключением одного значения, которое мы заменили выше на 75). Это кажется не логичным. Когда заполним пропуски по остальным столбцам, будем смотреть зависимости, имея в виду, что у нас есть 37 таких строчек.

Теперь посмотрим на пустые значения score:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.score))][['reason', 'traveltime', 'studytime', 'failures',
                                                      'schoolsup', 'paid', 'activities', 'higher',
                                                      'romantic', 'famrel', 'freetime', 'goout',
                                                      'Medu', 'Fedu', 'Pstatus', 'health', 'absences']]

Исходя из данных этой выборки, ничего особо не выделяется. Но проверим влияет ли failure на оценку по математике с помощью гистограммы распределения, возьмем failure от 1 до 3.

In [ ]:
stud_math.loc[stud_math.failures.between(1,3)].score.hist()

Критерий failure имеет такое же распределение как и общее score, где среднее~медиане одинаково. Значит заполним все непустые значения score значениями медианы:

In [ ]:
stud_math.score.fillna(stud_math.score.median(), inplace=True)

Проверим, остались ли пустые значенися в score:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.score))]

Посмотрим на итоговые данные в столбце score:

In [ ]:
stud_math.score.plot(kind='hist', grid=True, bins=20, title='Score')
stud_math.score.describe()

#### Studytime, granular

In [ ]:
get_describe('studytime, granular')

In [ ]:
stud_math['studytime, granular'].plot(
    kind='hist', grid=True, title='studytime, granular')

Посмотрим на таблицу с двумя столбцами вместе studytime и studytime, granular. Возможно они зависимы.

In [ ]:
stud_math[['studytime, granular', 'studytime']].head()

Как видно, столбец studytime, granular есть столбец studytime умноженный на -3. Проверим корреляцию:

In [ ]:
stud_math[['studytime, granular', 'studytime']].corr()

Корреляция = -1, поэтому можем смело убирать столбец из датафрейма:

In [ ]:
stud_math = stud_math.drop(columns='studytime, granular')

### Корреляция между числовыми признаками

In [ ]:
# Make new dataframe only with quantitative columns.
stud_math_num = stud_math[['age', 'absences', 'score']]
sns.pairplot(stud_math_num)

In [ ]:
stud_math_num.corr()

Построим тепловую карту распределения:

In [ ]:
sns.heatmap(stud_math_num.corr(), annot=True, cmap='coolwarm')

#### Выводы: 

Mы видим слабую корреляцию между количественными переменными. Можно наблюдать только, что пропусков занятий становится чуть больше с возрастом (появляются романтические отношения или иные цели в жизни). Пропуски занятий же практически не зависят от оценки, но судя по графикам все равно есть значения, где пропуски больше, тем оценка ниже.

Посмотрим теперь корреляцию, если бы мы убрали оценки равные 0 из датафрейма.

In [ ]:
score_notnull = stud_math.loc[stud_math.score != 0]
score_notnull_num = score_notnull[['age', 'absences', 'score']]
score_notnull_num.corr()

Теперь корреляция стала более явной у absences и score = -0.22: чем больше пропусков, тем ниже оценка. Это больше похоже на правду.

### Номинативные переменные

#### School

In [ ]:
get_describe('school')

#### Sex

In [ ]:
get_describe('sex')

#### Address

In [ ]:
get_describe('address')

Проверим взаимосвязь address с другими параметрами:

In [ ]:
get_relation(stud_math, 'address', 0)

Самая проглядывающая зависимость c traveltime. Давайте заменим на U, где traveltime=1,2, на R, где traveltime=3,4.

In [ ]:
# Create dataframe with nan address and traveltime.
df_ad = stud_math.iloc[np.where(pd.isnull(stud_math.address))][[
    'address', 'traveltime']]
df_ad_keys = df_ad['address'].keys()  # Extract indexes.
display(df_ad_keys)

travel_val = df_ad['traveltime'].values
l = len(df_ad_keys)

for i in range(0, l):
    if (travel_val[i] == 1) | (travel_val[i] == 2):
        stud_math.loc[df_ad_keys[i], 'address'] = 'U'
    elif (travel_val[i] == 3) | (travel_val[i] == 4):
        stud_math.loc[df_ad_keys[i], 'address'] = 'R'

In [ ]:
# Checking nan address values.
stud_math.iloc[np.where(pd.isnull(stud_math.address))][['school', 'reason', 'traveltime', 'studytime',
                                                        'schoolsup', 'paid', 'activities', 'higher',
                                                        'romantic', 'famrel', 'freetime', 'Medu',
                                                        'Fedu', 'Mjob', 'health', 'absences', 'score']]

Мы видим, что Medu = 4, тогда address = U. У третьего студента Medu=2, тоже U. Заменим:

In [ ]:
stud_math.address.fillna('U', inplace=True)

#### Famsize - family size

In [ ]:
get_describe('famsize')

Посмотрим, с какими столбцами есть взаимосвязь у famsize с помощью функции из предобработки:

In [ ]:
get_relation(stud_math, 'famsize', 0)

Как мы видим из таблиц связей, в каждом столбце модой является GT3. Заполним пропуски этим значением:

In [ ]:
stud_math.famsize.fillna('GT3', inplace=True)

#### Medu, Fedu - mother and father education

In [ ]:
get_describe('Medu')
get_describe('Fedu')

Значение 40 в Fedu явно ошибка, должно быть 4, исправим:

In [ ]:
stud_math.Fedu = stud_math.Fedu.replace(40, 4)

Посмотрим на пустые значения в образовании матери и отца:

In [ ]:
# Nan value in Mother education.
stud_math.iloc[np.where(pd.isnull(stud_math.Medu))][['reason', 'traveltime', 'studytime', 'failures',
                                                     'schoolsup', 'paid', 'activities', 'higher',
                                                     'romantic', 'famrel', 'freetime', 'goout',
                                                     'Fedu', 'nursery', 'health', 'absences', 'score']]

In [ ]:
# Nan value in Father education.
stud_math.iloc[np.where(pd.isnull(stud_math.Fedu))][['reason', 'traveltime', 'studytime', 'failures',
                                                     'schoolsup', 'paid', 'activities', 'higher',
                                                     'romantic', 'famrel', 'freetime', 'goout',
                                                     'Medu', 'nursery', 'health', 'absences', 'score']][:10]

Если есть зависимость между образованиями родителей, возможно нам удастся заполнить пропуски. Проверим, есть ли зависимости у столбцов образований родителей с другими, построим на примере Medu:

In [ ]:
get_relation(stud_math, 'Medu', 0)

Вышло очень много зависимостей, которые нам понадобятся для дальнейших заполнений пропусков других категориалных столбцов. Как мы и надеялись, образование родителей схожи. Эта таблица сводная таблица нам поможет в заполнении пропусков. Сначала заполним образование матери (там меньше всего пропусков), посмотрим на эту выборку еще раз:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.Medu))][['reason', 'traveltime', 'studytime', 'failures',
                                                     'schoolsup', 'paid', 'activities', 'higher',
                                                     'romantic', 'famrel', 'freetime', 'goout',
                                                     'Fedu', 'nursery', 'health', 'absences', 'score']]

Итак, когда образование отца:
- равно 1, самое частое образование матери по сводной таблице это тоже 1 (36 значений)
- равно 3, самое частое значение Medu = 4 (39 значений)
- равно 2, то самое частое значение Medu = 2 (47 значений)

Таким образом заполним эти пропущенне значения Medu модами по значениям Fedu:

In [ ]:
stud_math.loc[92, 'Medu'] = 1
stud_math.loc[193, 'Medu'] = 4
stud_math.loc[239, 'Medu'] = 2

Теперь аналогичным способом заменим пропуски Fedu по моде значений Medu (по нашей сводной таблице полученной выше). Для этого нам потребуются индексы и значения Medu.

In [ ]:
edu_keys = stud_math.iloc[np.where(pd.isnull(stud_math.Fedu))]['Medu'].keys()
medu_values = stud_math.iloc[np.where(
    pd.isnull(stud_math.Fedu))]['Medu'].values
l = len(edu_keys)

for i in range(0, l):
    if medu_values[i] == 1:
        stud_math.loc[edu_keys[i], 'Fedu'] = 1
    elif medu_values[i] == 2:
        stud_math.loc[edu_keys[i], 'Fedu'] = 2
    elif medu_values[i] == 3:
        stud_math.loc[edu_keys[i], 'Fedu'] = 3
    elif medu_values[i] == 4:
        stud_math.loc[edu_keys[i], 'Fedu'] = 4

Проверим остались ли пустые значения Fedu:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.Fedu))][['reason', 'traveltime', 'studytime', 'failures',
                                                     'schoolsup', 'paid', 'activities', 'higher',
                                                     'romantic', 'famrel', 'freetime', 'goout',
                                                     'Medu', 'nursery', 'health', 'absences', 'score']]

#### Mjob, Fjob - mother and father job

In [ ]:
get_describe('Mjob')
get_describe('Fjob')

Посмотрим, от каких столбцов зависит работа родителей на примере Mjob:

In [ ]:
get_relation(stud_math, 'Mjob', 0)

Возьмем уже заполненные столбцы и самые логичные для связи - это Medu и Fedu. Посмотрим на эти две сводбные таблицы:

In [ ]:
# Make a pivot table.
pivot_mj_me = stud_math.pivot_table(values=['score'], index=['Mjob'], columns=['Medu'],
                                    aggfunc='count', fill_value=0)
display(pivot_mj_me)
pivot_mj_fe = stud_math.pivot_table(values=['score'], index=['Mjob'], columns=['Fedu'],
                                    aggfunc='count', fill_value=0)
display(pivot_mj_fe)

Посмотрим, какие значения Medu, Fedu будут в выборке пустых значений Mjob:

In [ ]:
display(stud_math.iloc[np.where(pd.isnull(stud_math.Mjob))]
        ['Medu'].value_counts())
display(stud_math.iloc[np.where(pd.isnull(stud_math.Mjob))]
        ['Fedu'].value_counts())

Нулевых образований нет. По первой таблице можно сделать вывод, что при Medu=2 мода=46 - other, при Medu=4 мода=52 - teacher. Не очень ясно, какие варианты выбрать для Medu=1 (at_home или other) и Medu=3 (other или services). В этом случае посмотрим на вторую таблицу c Fedu.

- Medu=1: Fedu=1-4, то other (38-other > 22-at_home и тд)

- Medu=3: Fedu=1-3, то other, если Fedu=4, то service (27-services > 16-other).

В нашей выборке пропусков Mjob надо найти все пары (Medu, Fedu), где Medu = 1 и 3, и заполнить пропуски по вышеуказанному алгоритму.

In [ ]:
# Create dataframe with nan Mjob and two columns Medu, Fedu.
df_mjob = stud_math.iloc[np.where(pd.isnull(stud_math.Mjob))][[
    'Mjob', 'Fedu', 'Medu']]
df_mjob_keys = df_mjob['Mjob'].keys()  # Extract indexes.
display(df_mjob_keys)

Medu_val = df_mjob['Medu'].values
Fedu_val = df_mjob['Fedu'].values
l = len(df_mjob_keys)

for i in range(0, l):
    if (Medu_val[i] == 1) | (Medu_val[i] == 2):
        stud_math.loc[df_mjob_keys[i], 'Mjob'] = 'other'
    elif Medu_val[i] == 4:
        stud_math.loc[df_mjob_keys[i], 'Mjob'] = 'teacher'
    elif Medu_val[i] == 3:
        if Fedu_val[i] in [1, 2, 3]:
            stud_math.loc[df_mjob_keys[i], 'Mjob'] = 'other'
        else:
            stud_math.loc[df_mjob_keys[i], 'Mjob'] = 'services'

Теперь убедимся, что у Fjob тоже есть зависимости от Medu, Fedu:

In [ ]:
get_relation(stud_math, 'Fjob', 0)

Аналогично выведем сводные таблицы взаимосвязи Fjob с Medu, Fedu:

In [ ]:
# Make a pivot table.
pivot_fj_fe = stud_math.pivot_table(values=['score'], index=['Fjob'], columns=['Fedu'],
                                    aggfunc='count', fill_value=0)
display(pivot_fj_fe)
pivot_fj_me = stud_math.pivot_table(values=['score'], index=['Fjob'], columns=['Medu'],
                                    aggfunc='count', fill_value=0)
display(pivot_fj_me)

Смотря на первую таблицы мы видим, что везде модой является строка other, но при Fedu=4 все-таки есть еще и teacher. Если взять вспомогательную таблицу с Medu, то мы видим, что тут other является безоговорочной модой, поэтому мы примем Fjob везде = other. Сделаем замену:

In [ ]:
stud_math.Fjob.fillna('other', inplace=True)

#### Famrel - family relation

In [ ]:
get_describe('famrel')

Исходя из описания признака значение -1 являтеся ошибочным, заменим его на 1:

In [ ]:
stud_math.famrel = stud_math.famrel.replace(-1, 1)

Теперь проверим зависимости столбца famrel:

In [ ]:
get_relation(stud_math, 'famrel', 0)

Зависимостей нет, так как мода 4.0 сильно выделяется от остальных значений и равна среднему, заменим пропуски на 4:

In [ ]:
stud_math.famrel.fillna(4, inplace=True)

#### Pstatus - parents status

In [ ]:
get_describe('Pstatus')

Проверим зависимости столбца Pstatus с остальными в датафрейме:

In [ ]:
get_relation(stud_math, 'Pstatus', 0)

Только с Medu статус проживания родителей имеет связь, но как мы видим при любом значении Medu модой является значение T. Таким образом заменим пропуски на Т:

In [ ]:
stud_math['Pstatus'].fillna('T', inplace=True)

#### Reason

In [ ]:
get_describe('reason')

Посмотрим, есть ли зависимости причины выбора школы от каких-либо других показателей:

In [ ]:
get_relation(stud_math, 'reason', 0)

Получили достаточно интересные зависимости, но ни одной из них не хватит заполнить разом все пропуски. Самая подходящая кажется studytime: 

- если studytime=1, то выбираем course
- если studytime=3, то выбираем course
- если studytime=4, то выбираем reputation
- если studytime=2, то нам нужно выбрать из трех значений course, home, reputation (они имют примерн одинаковые значения по сводной таблице)

Тепер давайте посмотрим на выборку пустых значений reason, где studytime=2:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.reason))][['school', 'traveltime', 'studytime', 'failures',
                                                       'schoolsup', 'paid', 'activities', 'higher',
                                                       'romantic', 'famrel', 'freetime', 'Mjob',
                                                       'Medu', 'Fedu', 'health', 'absences', 'score']].loc[
    stud_math.studytime == 2
]

При studytime=2 смотрим на Mjob, если:

- Mjob=at_home, то выбираем course,
- Mjob=other, то выбираем home,
- Mjob=services, то выбираем course.

In [ ]:
# Create dataframe with nan reason and two columns: studytime and Mjob.
df_reason = stud_math.iloc[np.where(pd.isnull(stud_math.reason))][[
    'reason', 'studytime', 'Mjob']]
df_reason_keys = df_reason['reason'].keys()  # Extract indexes.
display(df_reason_keys)

study_val = df_reason['studytime'].values
Mjob_val = df_reason['Mjob'].values
l = len(df_reason_keys)

for i in range(0, l):
    if (study_val[i] == 1) | (study_val[i] == 3):
        stud_math.loc[df_reason_keys[i], 'reason'] = 'course'
    elif study_val[i] == 4:
        stud_math.loc[df_reason_keys[i], 'reason'] = 'reputation'
    elif study_val[i] == 2:
        if Mjob_val[i] == 'other':
            stud_math.loc[df_reason_keys[i], 'reason'] = 'home'
        else:
            stud_math.loc[df_reason_keys[i], 'reason'] = 'course'

Проверим, остались ли пропуски в reason:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.reason))][['school', 'traveltime', 'studytime', 'failures',
                                                       'schoolsup', 'paid', 'activities', 'higher',
                                                       'romantic', 'famrel', 'freetime', 'Mjob',
                                                       'Medu', 'Fedu', 'health', 'absences', 'score']]

Остался один пропуск из-за того, что studytime не заполнен. Но мы видим, что Mjob=at_home, тогда заменим это пустое значение reason на course:

In [ ]:
stud_math['reason'].fillna('course', inplace=True)

#### Guardian

In [ ]:
get_describe('guardian')

Проверим, есть ли зависимости у столбца guardian:

In [ ]:
get_relation(stud_math, 'guardian', 0)

По всем сводным таблицам зависимостей мы видим, что модой является мать. Заменим все пропуски на mother:

In [ ]:
stud_math['guardian'].fillna('mother', inplace=True)

#### Traveltime

In [ ]:
get_describe('traveltime')

Посмотрим на зависимости с traveltime:

In [ ]:
get_relation(stud_math, 'traveltime', 0)

Все зависимости кроме одной подказывают, что мода traveltime=1, поэтому возьмем за основу зависимость с выбором школы - school. Если школы GP, то traveltime=1, если школа MS, то выбираем traveltime=2.

In [ ]:
# Create dataframe with nan traveltime and school.
df_travel = stud_math.iloc[np.where(pd.isnull(stud_math.traveltime))][[
    'traveltime', 'school']]
df_travel_keys = df_travel['traveltime'].keys()  # Extract indexes.
display(df_travel_keys)

school_val = df_travel['school'].values
l = len(df_travel_keys)

for i in range(0, l):
    if school_val[i] == 'GP':
        stud_math.loc[df_travel_keys[i], 'traveltime'] = 1
    elif school_val[i] == 'MS':
        stud_math.loc[df_travel_keys[i], 'traveltime'] = 2

#### Studytime

In [ ]:
get_describe('studytime')

Пустых значений всего 7, посмотрим на них:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.studytime))][['reason', 'sex', 'traveltime', 'failures',
                                                          'schoolsup', 'paid', 'activities', 'higher',
                                                          'romantic', 'famrel', 'freetime', 'goout',
                                                          'Medu', 'Fedu', 'health', 'absences', 'score']]

Теперь посмотрим на зависимости studytime:

In [ ]:
get_relation(stud_math, 'studytime', 0)

Как видно из таблиц мода везде studytime=2 (самая читаемая зависимость у goout).  Заменим пропуски на 2:

In [ ]:
stud_math['studytime'].fillna(2, inplace=True)

#### Failures

In [ ]:
get_describe('failures')

Посмотрим, есть ли зависимости у промахов:

In [ ]:
get_relation(stud_math, 'failures', 0)

Исходя из таблиц зависимостей везде мода failure = 0. Заменим пропуски на нули:

In [ ]:
stud_math['failures'].fillna(0, inplace=True)

#### Schoolsup - school support

In [ ]:
get_describe('schoolsup')

Посмотрим, есть ли зависимости у столбца schoolsup:

In [ ]:
get_relation(stud_math, 'schoolsup', 0)

По всем показателям мода schoolsup=no. Сделаем замену:

In [ ]:
stud_math.schoolsup.fillna('no', inplace=True)

#### Activities

In [ ]:
get_describe('activities')

Посмотрим на эти пропуски:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.activities))][['reason', 'school', 'sex', 'failures',
                                                           'paid', 'schoolsup', 'famsup', 'higher',
                                                           'romantic', 'famrel', 'studytime', 'freetime', 'goout',
                                                           'Medu', 'Fedu', 'health', 'absences', 'score']]

Не думаю, что те студенты, которые не хотят поступать в университет, будут посещать доп занятия. Поставим студентам 160 и 163 - no.

In [ ]:
stud_math.loc[[160, 163], 'activities'] = 'no'

Проверим, какие зависимости у activities:

In [ ]:
get_relation(stud_math, 'activities', 0)

Исходя из полученных данных, выберем зависимость с Medu. Но значения в ней очень близки. Посмотрим какой p value имеет зависимость activities c Fedu:

In [ ]:
get_chi2(stud_math, 'activities', 'Fedu', 0.05, 1)

Мы видим, что значение p value достаточно близко к альфа=0.05. Возможно если бы все было заполненно верно, зависимость была более явной. Сделаем допущение и рассмотрим также зависимость с Fedu.

Построим сводную таблицу по activities в разрезе Medu и Fedu (все значения по Fedu и Medu мы ранее заполнили, поэтому данные будут полными):

In [ ]:
# Убираем нулевые значения Medu и Fedu.
stud_math_edu = stud_math.loc[(stud_math.Medu != 0) & (stud_math.Fedu != 0)]

pivot_act_edu = stud_math_edu.pivot_table(values=['score'], index=['activities', 'Fedu'], columns=['Medu'],
                                          aggfunc='count', fill_value=0)
display(pivot_act_edu)

Теперь пробежимся по парам (Fedu, Medu) в выборке пустых activities и заменим на yes, либо no, исходя из сводной таблицы выше: какое из значений на пересечении указанной пары (Fedu, Medu) больше, то и выбираем либо yes, либо no. Сначала создадим матрицу 4 на 4 со значениями либо yes, либо no, исходя из нашей сводной таблицы. Допустим, что если значение в сводной таблице  в no будет >= чем в yes, то мы выберем activity = no.

In [ ]:
display(pivot_act_edu.xs(('yes', 1))[1])
display(pivot_act_edu.xs(('no', 1))[1])

In [ ]:
# Create empty activity matrix (4, 4).
act_df = pd.DataFrame(index=[1, 2, 3, 4], columns=[1, 2, 3, 4])

# Now we will fill this matrix with 'no' and 'yes' depending on pivot_act_edu.
for i in range(1, 5):
    for j in range(0, 4):
        if pivot_act_edu.xs(('no', i))[j] >= pivot_act_edu.xs(('yes', i))[j]:
            act_df[j+1][i] = 'no'
        else:
            act_df[j+1][i] = 'yes'

display(act_df)

In [ ]:
# Create dataframe with nan activities and two columns Medu, Fedu.
df_act = stud_math.iloc[np.where(pd.isnull(stud_math.activities))][[
    'activities', 'Fedu', 'Medu']]
df_act_keys = df_act['activities'].keys()  # Extract indexes.
display(df_act_keys)

Medu_val = df_act['Medu'].values
Fedu_val = df_act['Fedu'].values
l = len(df_act_keys)

for k in range(0, l):
    for i in range(1, 5):  # strings of act_df
        for j in range(1, 5):  # columns of act_df
            if (Fedu_val[k] == i) & (Medu_val[k] == j):
                stud_math.loc[df_act_keys[k], 'activities'] = act_df[j][i]

Проверим, остались ли пропуски в activitites:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.activities))]

#### Famsup - family support

In [ ]:
get_describe('famsup')

Посмотрим на зависимости famsup:

In [ ]:
get_relation(stud_math, 'famsup', 0)

Самыми подходящими зависимостями видятся Medu, Fedu. При их значениях всех, кроме 1, мода famsup = yes. Когда Medu и Fedu оба равны = 1, то no. Вопросом лишь остаются пары, где кто-то один из Medu, Fedu имеет 1, а второй другое значение. Посмотрим на эту выборку:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.famsup))][['school', 'sex', 'failures',
                                                       'paid', 'schoolsup', 'famsup', 'higher',
                                                       'romantic', 'famrel', 'studytime', 'freetime', 'goout',
                                                       'Medu', 'Fedu', 'Mjob', 'Fjob', 'absences', 'score']].loc[
    (stud_math.Medu == 1) | (stud_math.Fedu == 1)
]

Мы видим 5 одинаковых пар (2, 1), где непонятно выбрать yes или no (остальные будут no), посмотрим их отдельно:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.famsup))][['school', 'sex', 'failures',
                                                       'paid', 'schoolsup', 'famsup', 'higher',
                                                       'romantic', 'famrel', 'studytime', 'freetime', 'goout',
                                                       'Medu', 'Fedu', 'Mjob', 'Fjob', 'absences', 'score']].loc[
    (stud_math.Medu == 2) & (stud_math.Fedu == 1)
]

Из другой зависимости от пола, девочкам больше дают поддержку, нежели мальчикам, поэтому поставим yes девочкам, останутся два мальчика. Студент 78 имеет не хочет получать высшее, поэтому по таблице с higher famsup = no. У второго мальчика higher = nan, но исодя из работы матери (services) можем взять значение из этой сводной таблицы и увидеть, что там есть мода famsup = yes.

Таким образом, пропуски в famsup заменяем на yes, кроме этих студентов:
- 78 (M c higher=no)
- 127 (Medu+Fedu = (0,1))
- 61, 234, 272, 283 (Medu+Fedu = (1,1))

In [ ]:
stud_math.loc[[78, 127, 61, 234, 272, 283], 'famsup'] = 'no'

In [ ]:
stud_math.famsup.fillna('yes', inplace=True)

In [ ]:
# Checking nan values in famsup.
stud_math.iloc[np.where(pd.isnull(stud_math.famsup))]

#### Paid

In [ ]:
get_describe('paid')

Посмотрим на зависимости столбца paid:

In [ ]:
get_relation(stud_math, 'paid', 0)

Наилучшими вариантами таблиц данных для заполнения пропусков в paid являются Medu, Mjob.
При Medu = 1,2,3 - no. Посмотрим на выборку пропущенных paid, где Medu=0,4:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.paid))][['reason', 'sex', 'failures',
                                                     'famsup', 'schoolsup', 'famsup', 'higher',
                                                     'romantic', 'famrel', 'studytime', 'freetime', 'goout',
                                                     'Medu', 'Fedu', 'Mjob', 'Fjob', 'absences', 'score']].loc[
    (stud_math.Medu == 0) | (stud_math.Medu == 4)
]

Medu=0 нет, остаются = 4. Все ли paid будут по этой выборке = yes? Посмотрим на работу матери: при Mjob= other или home, paid = no (по сводной таблице зависимости выше). У нас есть только один студент с Mjob=other. Заменим у него paid на no:

In [ ]:
stud_math.loc[384, 'paid'] = 'no'

Остается заменить paid на no, когда Medu=1,2,3, и на yes, когда Medu=4:

In [ ]:
# Create dataframe with nan paid and Medu.
df_paid = stud_math.iloc[np.where(pd.isnull(stud_math.paid))][['paid', 'Medu']]
df_paid_keys = df_paid['paid'].keys()  # Extract indexes.
display(df_paid_keys)

Medu_val = df_paid['Medu'].values
l = len(df_paid_keys)

for i in range(0, l):
    if Medu_val[i] == 4:
        stud_math.loc[df_paid_keys[i], 'paid'] = 'yes'
    else:
        stud_math.loc[df_paid_keys[i], 'paid'] = 'no'

In [ ]:
# Checking nan values in paid.
stud_math.iloc[np.where(pd.isnull(stud_math.paid))]

#### Nursery

In [ ]:
get_describe('nursery')

Посмотрим на зависимости признака nursery:

In [ ]:
get_relation(stud_math, 'nursery', 0)

По всем зависимым признакам из таблиц мы видим, что мода nursery везде равна yes. Заменим пустые значения на yes:

In [ ]:
stud_math.nursery.fillna('yes', inplace=True)

#### Higher

In [ ]:
get_describe('higher')

In [ ]:
# Check dependence with other variables.
get_relation(stud_math, 'higher', 0)

Как мы видим из всех таблиц модой является значение yes. Заменим все пропуски:

In [ ]:
stud_math.higher.fillna('yes', inplace=True)

#### Internet

In [ ]:
get_describe('internet')

In [ ]:
# Check dependence with other variables.
get_relation(stud_math, 'internet', 0)

Как мы видим из всех таблиц модой является значение yes. Заменим все пропуски:

In [ ]:
stud_math.internet.fillna('yes', inplace=True)

#### Romantic

In [ ]:
get_describe('romantic')

In [ ]:
# Check dependence with other variables.
get_relation(stud_math, 'romantic', 0)

Исходя из трех таблиц мода romantic = no, кроме одного значения в таблице с higher: когда higher = no, то romantic скорее всего равно yes. Посмотрим на выборку пустых romantic, где higher = no:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.romantic))][['reason', 'sex', 'failures',
                                                         'famsup', 'schoolsup', 'famsup', 'higher',
                                                         'famrel', 'studytime', 'freetime', 'goout',
                                                         'Medu', 'Fedu', 'Mjob', 'Fjob', 'absences', 'score']].loc[
    stud_math.higher == 'no'
]

Всего лишь одна строка такая, заметим, что score=0, freetime = 5(максимум). Проставим этому студенту romantic=yes, а остальные пропуски заменим на no:

In [ ]:
stud_math.loc[239, 'romantic'] = 'yes'

In [ ]:
stud_math.romantic.fillna('no', inplace=True)

#### Freetime

In [ ]:
get_describe('freetime')

Посмотрим на пропуски:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.freetime))][['reason', 'sex', 'failures',
                                                         'famsup', 'schoolsup', 'famsup', 'higher',
                                                         'famrel', 'studytime', 'goout', 'romantic',
                                                         'Medu', 'Fedu', 'Mjob', 'Fjob', 'absences', 'score']]

In [ ]:
# Check dependence with other variables.
get_relation(stud_math, 'freetime', 0)

Самая очевидная зависимость freetime c goout. В нашей выборке goout=1-4. При goout=1, мы получаем только одного студента - девушку с schoolsup=no, по обоим признакам мода = 3, заменим freetime у студента 311 на 3:

In [ ]:
stud_math.loc[311, 'freetime'] = 3

При goout=2-3 мода freetime = 3, goout=4 => freetime=4. Сделаем соответствующие замены:

In [ ]:
# Create dataframe with nan freetime and goout.
df_free = stud_math.iloc[np.where(pd.isnull(stud_math.freetime))][[
    'freetime', 'goout']]
df_free_keys = df_free['freetime'].keys()  # Extract indexes.
display(df_free_keys)

goout_val = df_free['goout'].values
l = len(df_free_keys)

for i in range(0, l):
    if goout_val[i] == 4:
        stud_math.loc[df_free_keys[i], 'freetime'] = 4
    else:
        stud_math.loc[df_free_keys[i], 'freetime'] = 3

In [ ]:
# Check nan values in freetime.
stud_math.iloc[np.where(pd.isnull(stud_math.freetime))]

#### Goout

In [ ]:
get_describe('goout')

Посмотрим на пропущенные значения goout:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.goout))][['sex', 'failures', 'traveltime',
                                                      'famsup', 'schoolsup', 'famsup', 'higher',
                                                      'famrel', 'studytime', 'freetime', 'romantic',
                                                      'Medu', 'Fedu', 'Mjob', 'Fjob', 'absences', 'score']]

In [ ]:
# Check dependence with other variables.
get_relation(stud_math, 'goout', 0)

По логике самой очевидной зависимостью является столбец freetime. При freetime равным 2,3,4,5 мы получаем, что goout равен 2,3,4,5. Если freetime=1, мы видим из выборки одного такого студента: studytime=2, traveltime=1 отсюда мода goout=3. Сделаем соответствующие замены:

In [ ]:
stud_math.loc[89, 'goout'] = 3

In [ ]:
# Create dataframe with nan goout and freetime.
df_goout = stud_math.iloc[np.where(pd.isnull(stud_math.goout))][[
    'goout', 'freetime']]
df_goout_keys = df_goout['goout'].keys()  # Extract indexes.
display(df_goout_keys)

freetime_val = df_goout['freetime'].values
l = len(df_goout_keys)

for i in range(0, l):
    if freetime_val[i] == 2:
        stud_math.loc[df_goout_keys[i], 'goout'] = 2
    elif freetime_val[i] == 3:
        stud_math.loc[df_goout_keys[i], 'goout'] = 3
    elif freetime_val[i] == 4:
        stud_math.loc[df_goout_keys[i], 'goout'] = 4
    elif freetime_val[i] == 5:
        stud_math.loc[df_goout_keys[i], 'goout'] = 5

In [ ]:
# Checking nan values in goout.
stud_math.iloc[np.where(pd.isnull(stud_math.goout))]

Столбцы freetime и goout прямозависимы, но какой из них удалить, пока не ясно, так как они имеют зависимости с разными признаками в датафрейме. Позже посмотрим на зависимости их обоих с целевой переменной score.

#### Health

In [ ]:
get_describe('health')

In [ ]:
# Check dependence with other variables.
get_relation(stud_math, 'health', 0)

Интересные зависимости, посмотрим на выборку пустых значений с этими столбцами:

In [ ]:
stud_math.iloc[np.where(pd.isnull(stud_math.health))][['reason', 'sex', 'failures', 'traveltime',
                                                       'famsup', 'higher', 'studytime', 'freetime',
                                                       'famrel', 'romantic', 'guardian',
                                                       'Medu', 'Fedu', 'Mjob', 'Fjob', 'absences', 'score']]

По всем данным почти везде мода health=5. Лишь в guardian=other она равна 3, но такого значения guardian в нашей выборке нет. При studytime=4 мода health отлична от 5, она равна 3, заменим у троих таких студентов health на 3:

In [ ]:
stud_math.loc[[256, 259, 330], 'health'] = 3

Остальные значения заменим на health = 5:

In [ ]:
stud_math.health.fillna(5, inplace=True)

In [ ]:
# Checking nan values in goout.
stud_math.iloc[np.where(pd.isnull(stud_math.health))]

### Анализ номинативных переменных

Посмотрим различаются ли распределения оценки score в зависимости от значений в номинативных переменных. Перечислим эти переменные:

In [ ]:
stud_math.columns.drop(['age', 'score', 'absences'])

In [ ]:
column_list = stud_math.columns.drop(['age', 'score', 'absences']).tolist()

Посмотрим на графики распределений с помощью boxplot и функции из предобработки:

In [ ]:
for col in column_list:
    get_boxplot(stud_math, col)

Исходя из графиков, есть большая вероятность, что значимыми столбцами будут: address, Medu, Fedu, Mjob, studytime, failures, schoolsup, paid, higher, goout.

Попробуем построить графики для датафрейма, где оценка по мат-ке не равна 0 (как мы помним таких строчек было 37). Для этого создадим отдельный датафрейм, где оценка не равна 0:

In [ ]:
score_notnull = stud_math.loc[stud_math.score != 0]

In [ ]:
# Create boxplots with dataframe where score is not null.
for col in column_list:
    get_boxplot(score_notnull, col)

По этим боксплотам значимыми столбцами могут быть: address, sex, Medu, Fedu, Mjob, Fjob, studytime, failures, schoolsup, higher, goout. Отличия немного есть.

Теперь посмотрим, есть ли статистическая разница в распределении оценок по номинативным признакам с помощью теста Стьюдента. Проверим нулевую гипотезу о том, что распределения оценок по математике по различным параметрам неразличимы:

In [ ]:
for col in column_list:
    get_ttest(stud_math, col)

Проверим теперь отдельно, изменятся ли итоговые столбцы, если оценка не равна 0:

In [ ]:
for col in column_list:
    get_ttest(score_notnull, col)

Как мы видим пересечение имеется, но всего по 5 столбцам: address, Medu, Fedu, Mjob, failures. 

Столбцы, которые отличаются, все по здравому смыслу влияют на оценку студентов. Заметим также, что во втором тесте p value по общим параметрам получились даже лучше. Итак, возьмем объединие двух получившихся множеств показателей.

### Выводы: 

- все пустые значения были заполнены, данные стали полными.
- выбросы были найдены только в графе "age" (22 года) и "absences" (больше 25), но все значения оставлены из здравого смысла.
- отрицательная корреляция между score и age говорит нам о том, что с возрастом оценка ухудшается, а отрицательная корреляция score c absences - о том, что чем больше пропусков, тем ниже оценка. Оба вывода вполне логичны. 
-  самые важные параметры, которые мы оставим для построения дальнейшей модели - это age, absences, address, Medu, Fedu, Mjob, Fjob, failures, paid, higher, romantic, studytime, schoolsup, goout.